In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
import xgboost
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [ ]:
import shap
shap.initjs()

In [ ]:
import random
random.seed(999)
np.random.seed(999)

In [ ]:
def build_modeling_data(fcpo_data_daily,fcpo_feats,target_label='lprofit_ind',
                        train_start='2015-01-01',train_end='2017-12-31',test_start='2018-01-01',test_end='2018-10-01'):
    fcpo_train=fcpo_feats[train_start:train_end]
    fcpo_test = fcpo_feats[test_start:test_end]

    fcpo_train=fcpo_train.merge(fcpo_data_daily[['Open','prev_open_change_pct',target_label]],left_index=True,right_index=True)
    fcpo_traindata=fcpo_train.iloc[:,0:-1]
    fcpo_trainlabels=fcpo_train.iloc[:,-1:]
    
    fcpo_test=fcpo_test.merge(fcpo_data_daily[['Open','prev_open_change_pct',target_label]],left_index=True,right_index=True)
    fcpo_testdata=fcpo_test.iloc[:,0:-1]
    fcpo_testlabels=fcpo_test.iloc[:,-1]
    return fcpo_traindata,fcpo_trainlabels,fcpo_testdata,fcpo_testlabels

def buil_cross_validated_model(ml_model,traindata,trainlabels,tsplit,scoring_metric='roc_auc'):
    ml_cval_scores=cross_val_score(ml_model,traindata,np.ravel(trainlabels),
                                       scoring=scoring_metric,cv=tsplit)
    ml_model.fit(traindata,np.ravel(trainlabels))
    return ml_model,ml_cval_scores

def derive_classification_labels(x,threshold=85):
    clf_thr=np.percentile(x[:,1],q=threshold)
    clf_labels=list(map(lambda x: 1 if x > clf_thr else 0, x[:,1]))
    return clf_labels

def zscore_func_improved(x,window_size=20):
    rolling_mean=x.rolling(window=window_size).mean().bfill()
    rolling_std = x.rolling(window=window_size).std().bfill()
    return (x-rolling_mean)

## Loading Preprocessed Data 


In [ ]:
fcpo_daily_badjusted=pd.read_hdf('data/processed_dta.h5','fcpo_daily_badjusted')
fcpo_hourly_badjusted=pd.read_hdf('data/processed_dta.h5','fcpo_hourly_badjusted')
fcpo_badjusted_tind=pd.read_hdf('data/processed_dta.h5','fcpo_badjusted_tind')
fcpo_badjusted_cdlind=pd.read_hdf('data/processed_dta.h5','fcpo_badjusted_cdlind')
fcpo_badjusted_cind = pd.read_hdf('data/processed_dta.h5','fcpo_badjusted_cind')

fcpo_badjusted_feats=fcpo_badjusted_tind.merge(fcpo_badjusted_cdlind,left_index=True,right_index=True).merge(
                                                    fcpo_badjusted_cind,left_index=True,right_index=True)
#fcpo_data_daily[['Open','High','Low','Close']]=fcpo_data_daily[['Open','High','Low','Close']].apply(lambda x: zscore_func_improved(x))
## using the subset of data for training to exclude 2008 oil crisis
fcpo_badjusted_feats=fcpo_badjusted_feats['2008-01-01':'2017-01-01']

In [ ]:
## read the non back-adjusted data
fcpo_daily_nadjusted=pd.read_hdf('data/processed_dta.h5','fcpo_daily_nadjusted')
fcpo_nadjusted_tind=pd.read_hdf('data/processed_dta.h5','fcpo_nadjusted_tind')
fcpo_nadjusted_cdlind=pd.read_hdf('data/processed_dta.h5','fcpo_nadjusted_cdlind')

fcpo_nadjusted_feats=fcpo_nadjusted_tind.merge(fcpo_nadjusted_cdlind,left_index=True,right_index=True)
fcpo_nadjusted_feats=fcpo_nadjusted_feats['2011-01-01':'2018-10-01']

## Machine Learning models

### Time-series cross validation

In [ ]:
fcpo_tsplit=TimeSeriesSplit(n_splits=3)

### Long call trading model 

In [ ]:
fcpo_lprofit_traindata,fcpo_lprofit_trainlabels,fcpo_lprofit_testdata,fcpo_lprofit_testlabels=build_modeling_data(
                                fcpo_daily_nadjusted,fcpo_nadjusted_feats,'lprofit_ind','2015-01-01','2017-12-31','2018-01-01','2019-01-01')

## Auto ML Pipeline

In [ ]:
from sklearn.decomposition import FastICA
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.pipeline import make_pipeline

In [ ]:
lprofit_pipeline = make_pipeline(
    FastICA(tol=0.2),
    SelectPercentile(score_func=f_classif, percentile=37),
    RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.1, min_samples_leaf=9, min_samples_split=20, 
                           n_estimators=100,random_state=333)
)

In [ ]:
lprofit_pipeline.fit(fcpo_lprofit_traindata,np.ravel(fcpo_lprofit_trainlabels))

tpot_lprofit_test_predlabels=lprofit_pipeline.predict(fcpo_lprofit_testdata)
tpot_lprofit_test_proba=lprofit_pipeline.predict_proba(fcpo_lprofit_testdata)

### Random Search for Best Hyper Parameters

In [ ]:
## Use the RandomSearch funciton to find the best hyper parameters of Random forrests
rf_param_dist = {"max_depth": [5,10,20],
                  "min_samples_split": [2,4,6],
                  "min_samples_leaf": [1,2,4],
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}

xgb_param_dist={"n_estimators": [200,300,500],
                "max_depth": [5,10,20],
                "learning_rate": [0.01,0.05,0.1],
                "gamma": [0,0.01,0.05],
                "subsample": [0.5,0.8,1],
                "colsample_bytree": [0.5,0.8,1]  
}

In [ ]:
%%time
rf_lprofit_rsearch=RandomizedSearchCV(RandomForestClassifier(random_state=333,n_jobs=-1),
                                      rf_param_dist,cv=fcpo_tsplit,scoring='roc_auc',n_iter=100,n_jobs=-1)

rf_lprofit_rsearch=rf_lprofit_rsearch.fit(fcpo_lprofit_traindata,np.ravel(fcpo_lprofit_trainlabels))

In [ ]:
%%time
xgb_lprofit_rsearch=RandomizedSearchCV(xgboost.XGBClassifier(random_state=333),
                                      xgb_param_dist,cv=fcpo_tsplit,scoring='roc_auc',n_iter=100,n_jobs=-1)

xgb_lprofit_rsearch=xgb_lprofit_rsearch.fit(fcpo_lprofit_traindata,np.ravel(fcpo_lprofit_trainlabels))

#### Random Forrest

In [ ]:
### Compute cross-validation scores using time-series splits for a defined configuration ###

#rf_lprofit= RandomForestClassifier(n_estimators=300,max_depth=20,min_samples_split=2,
#                                  min_samples_leaf=2,criterion='gini',random_state=333)
#rf_lprofit,rf_lprofit_cval_scores=buil_cross_validated_model(rf_lprofit,fcpo_lprofit_traindata,fcpo_lprofit_trainlabels,fcpo_tsplit)

In [ ]:
rf_lprofit_test_proba=rf_lprofit_rsearch.best_estimator_.predict_proba(fcpo_lprofit_testdata)
rf_lprofit_test_predlabels=rf_lprofit_rsearch.best_estimator_.predict(fcpo_lprofit_testdata)
#rf_lprofit_test_predlabels=derive_classification_labels(rf_lprofit_test_proba)

#### XGBoost

In [ ]:
#xgb_lprofit=xgboost.XGBClassifier(objective='binary:logistic',max_depth=10,n_estimators=500,learning_rate=0.01,gamma=0.01,
#                                    subsample=0.8,colsample_bytree=0.8,n_jobs=-1,random_state=333)

#xgb_lprofit,xgb_lprofit_cval_scores=buil_cross_validated_model(xgb_lprofit,fcpo_lprofit_traindata,fcpo_lprofit_trainlabels,fcpo_tsplit)

In [ ]:
xgb_lprofit_test_proba=xgb_lprofit_rsearch.best_estimator_.predict_proba(fcpo_lprofit_testdata)
xgb_lprofit_test_predlabels=xgb_lprofit_rsearch.best_estimator_.predict(fcpo_lprofit_testdata)
#xgb_lprofit_test_predlabels=derive_classification_labels(xgb_lprofit_test_proba)

### Short call trading model 

In [ ]:
fcpo_sprofit_traindata,fcpo_sprofit_trainlabels,fcpo_sprofit_testdata,fcpo_sprofit_testlabels=build_modeling_data(fcpo_daily_nadjusted,
                                        fcpo_nadjusted_feats,'sprofit_ind','2015-01-01','2017-12-31','2018-01-01','2019-01-01')

In [ ]:
sprofit_pipeline = make_pipeline(
    FastICA(tol=0.2),
    SelectPercentile(score_func=f_classif, percentile=37),
    RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.1, min_samples_leaf=9, min_samples_split=20, n_estimators=100)
)

In [ ]:
sprofit_pipeline.fit(fcpo_sprofit_traindata,np.ravel(fcpo_sprofit_trainlabels))

tpot_sprofit_test_predlabels=sprofit_pipeline.predict(fcpo_sprofit_testdata)
tpot_sprofit_test_proba=sprofit_pipeline.predict_proba(fcpo_sprofit_testdata)

In [ ]:
%%time
rf_sprofit_rsearch=RandomizedSearchCV(RandomForestClassifier(random_state=333,n_jobs=-1),
                                      rf_param_dist,cv=fcpo_tsplit,scoring='roc_auc',n_iter=100,n_jobs=-1)

rf_sprofit_rsearch=rf_sprofit_rsearch.fit(fcpo_sprofit_traindata,np.ravel(fcpo_sprofit_trainlabels))

In [ ]:
%%time
xgb_sprofit_rsearch=RandomizedSearchCV(xgboost.XGBClassifier(random_state=333),
                                      xgb_param_dist,cv=fcpo_tsplit,scoring='roc_auc',n_iter=100,n_jobs=-1)

xgb_sprofit_rsearch=xgb_sprofit_rsearch.fit(fcpo_sprofit_traindata,np.ravel(fcpo_sprofit_trainlabels))

#### Random Forrest

In [ ]:
#rf_sprofit= RandomForestClassifier(n_estimators=300,max_depth=20,min_samples_split=2,
#                                  min_samples_leaf=2,criterion='entropy',random_state=333)
#rf_sprofit,rf_sprofit_cval_scores=buil_cross_validated_model(rf_sprofit,fcpo_sprofit_traindata,fcpo_sprofit_trainlabels,
#                                                                fcpo_tsplit)

In [ ]:
rf_sprofit_test_proba=rf_sprofit_rsearch.best_estimator_.predict_proba(fcpo_sprofit_testdata)
rf_sprofit_test_predlabels=rf_sprofit_rsearch.best_estimator_.predict(fcpo_sprofit_testdata)
#rf_sprofit_test_predlabels=derive_classification_labels(rf_sprofit_test_proba)

### XGBoost

In [ ]:
#xgb_sprofit=xgboost.XGBClassifier(objective='binary:logistic',max_depth=10,n_estimators=500,learning_rate=0.01,gamma=0.01,
#                                    subsample=0.8,colsample_bytree=0.8,n_jobs=-1,random_state=333)
#xgb_sprofit,xgb_sprofit_cval_scores=buil_cross_validated_model(xgb_sprofit,fcpo_sprofit_traindata,fcpo_sprofit_trainlabels,
#                                                                fcpo_tsplit)

In [ ]:
xgb_sprofit_test_proba=xgb_sprofit_rsearch.best_estimator_.predict_proba(fcpo_sprofit_testdata)
xgb_sprofit_test_predlabels=xgb_sprofit_rsearch.best_estimator_.predict(fcpo_sprofit_testdata)
#xgb_sprofit_test_predlabels=derive_classification_labels(xgb_sprofit_test_proba)

## Evaluation of Models

### Feature Importance for Long Profit Model

In [ ]:
xgb_lprofit_explainer=shap.TreeExplainer(xgb_lprofit_rsearch.best_estimator_)
xgb_lprofit_shapvalues=xgb_lprofit_explainer.shap_values(fcpo_lprofit_traindata)
shap.summary_plot(xgb_lprofit_shapvalues,fcpo_lprofit_traindata)

### Precision and Recall (Long Profit)

In [ ]:
print("RF precision : {pscore}, recall: {rscore} auc:{auc_score}".format(pscore=round(precision_score(fcpo_lprofit_testlabels,rf_lprofit_test_predlabels),2),
                                                     rscore=round(recall_score(fcpo_lprofit_testlabels,rf_lprofit_test_predlabels),2),
                                                     auc_score=round(roc_auc_score(fcpo_lprofit_testlabels,rf_lprofit_test_proba[:,1]),2)))

In [ ]:
print("XGB precision : {pscore}, recall: {rscore} auc: {auc_score}".format(pscore=round(precision_score(fcpo_lprofit_testlabels,xgb_lprofit_test_predlabels),2),
                                                      rscore=round(recall_score(fcpo_lprofit_testlabels,xgb_lprofit_test_predlabels),2),
                                                     auc_score=round(roc_auc_score(fcpo_lprofit_testlabels,xgb_lprofit_test_proba[:,1]),2)))

In [ ]:
print("TPOT precision : {pscore}, recall: {rscore} auc: {auc_score}".format(pscore=round(precision_score(fcpo_lprofit_testlabels,tpot_lprofit_test_predlabels),2),
                                                      rscore=round(recall_score(fcpo_lprofit_testlabels,tpot_lprofit_test_predlabels),2),
                                                     auc_score=round(roc_auc_score(fcpo_lprofit_testlabels,tpot_lprofit_test_proba[:,1]),2)))

### Feature Importance for Short Profit Model

In [ ]:
xgb_sprofit_explainer=shap.TreeExplainer(xgb_sprofit_rsearch.best_estimator_)
xgb_sprofit_shapvalues=xgb_sprofit_explainer.shap_values(fcpo_sprofit_traindata)
shap.summary_plot(xgb_sprofit_shapvalues,fcpo_sprofit_traindata)

### Precision and Recall (Short Profit)

In [ ]:
print("RF precision : {pscore}, recall: {rscore} auc:{auc_score}".format(
                                                    pscore=round(precision_score(fcpo_sprofit_testlabels,rf_sprofit_test_predlabels),2),
                                                    rscore=round(recall_score(fcpo_sprofit_testlabels,rf_sprofit_test_predlabels),2),
                                                    auc_score=round(roc_auc_score(fcpo_sprofit_testlabels,rf_sprofit_test_proba[:,1]),2)))

In [ ]:
print("XGB precision : {pscore}, recall: {rscore} auc:{auc_score}".format(pscore=round(precision_score(fcpo_sprofit_testlabels,xgb_sprofit_test_predlabels),2),
                                                      rscore=round(recall_score(fcpo_sprofit_testlabels,xgb_sprofit_test_predlabels),2),
                                                     auc_score=round(roc_auc_score(fcpo_sprofit_testlabels,xgb_sprofit_test_proba[:,1]),2)))

In [ ]:
print("TPOT precision : {pscore}, recall: {rscore} auc: {auc_score}".format(pscore=round(precision_score(fcpo_sprofit_testlabels,tpot_sprofit_test_predlabels),2),
                                                      rscore=round(recall_score(fcpo_sprofit_testlabels,tpot_sprofit_test_predlabels),2),
                                                     auc_score=round(roc_auc_score(fcpo_sprofit_testlabels,tpot_sprofit_test_proba[:,1]),2)))

## Financial Metrics

The final objective of this study is to improve the ROI, hence we are focused on creating a score-card to measure the investement metrics to select the final model,

- Total Returns
- Maximum Drawdown
- Number of Wins
- Number of Losses

**Calculation of Returns**

* When the sprofit_indicator is true, then return is (open_price - next_low_price) - (commission_price),
* Else when the sprofit_indicator is false, the model waits until the end of the day before selling it hence the return is (open_price - next_close_price) - (commission_price)


In [ ]:
def calculate_long_returns(x,profit_field,pred_field,commission_price,num_units,profit_thr=0.03,loss_thr=0.01):
    return_myr=0
    #when the prediction is zero, the model accumulates no returns
    if x[pred_field]==0:
        return_myr=0
    #when the model makes a prediction
    elif x[pred_field]==1:
        #when it is correct, sell it off when it reaches max x% profit zone
        if x[profit_field]==1:
            return_myr=((x['Open']*(1+profit_thr)-x['Open'])*num_units)-commission_price
        #when it went wrong, sell it off when it reaches x% loss or wait till it reaches eod
        elif x[profit_field]==0:
            max_loss_val=min(x['Low']-x['Open'],x['next_1low']-x['Open'])
            thr_loss_val=(x['Open']*(1-loss_thr))-x['Open']
            if max_loss_val < thr_loss_val:
                return_myr=thr_loss_val*num_units-commission_price
            else:
                return_myr= (x['next_1close']-x['Open'])*num_units-commission_price
    return return_myr

def calculate_short_returns(x,profit_field,pred_field,commission_price,num_units,profit_thr=0.03,loss_thr=0.01):
    return_myr=0
    #when the prediction is zero, the model accumulates no returns
    if x[pred_field]==0:
        return_myr=0
    #when the model makes a prediction
    elif x[pred_field]==1:
        if x[profit_field]==1:
        #when it is correct, sell it off when it reaches max x% profit zone
            return_myr=((x['Open']-x['Open']*(1-profit_thr))*num_units)-commission_price
        #when it went wrong, sell it off when it reaches x% loss or wait till it reaches eod
        elif x[profit_field]==0:
            max_loss_val=min(x['Open']-x['High'],x['Open']-x['next_1high'])
            thr_loss_val=x['Open']-(x['Open']*(1+loss_thr))
            if max_loss_val < thr_loss_val:
                return_myr=thr_loss_val*num_units-commission_price
            else:
                return_myr=(x['Open']-x['next_1close'])*num_units-commission_price
    return return_myr

def max_drawdown(X):
    mdd = 0
    peak = X[0]
    mdd_peak=0
    mdd_x=0
    for x in X:
        if x > peak: 
            peak = x
        dd = (peak - x) / peak
        if dd > mdd:
            mdd = dd
            mdd_peak=peak
            mdd_x=x
    return mdd_peak,mdd_x,mdd

In [ ]:
fcpo_eval_df=fcpo_daily_nadjusted[['Open','High','Low','next_1low','next_1high','next_1close',
                             'lprofit_ind','sprofit_ind','lprofit_ind_2pct','sprofit_ind_2pct']].merge(
    pd.DataFrame(rf_lprofit_test_predlabels,columns=['rf_lprofit_prediction'],index=fcpo_lprofit_testlabels.index),
            left_index=True,right_index=True).merge(
    pd.DataFrame(xgb_lprofit_test_predlabels,columns=['xgb_lprofit_prediction'],index=fcpo_lprofit_testlabels.index),
            left_index=True,right_index=True).merge(
    pd.DataFrame(rf_sprofit_test_predlabels,columns=['rf_sprofit_prediction'],index=fcpo_sprofit_testlabels.index),
            left_index=True,right_index=True).merge(
    pd.DataFrame(xgb_sprofit_test_predlabels,columns=['xgb_sprofit_prediction'],index=fcpo_sprofit_testlabels.index),
            left_index=True,right_index=True).merge(
    pd.DataFrame(tpot_lprofit_test_predlabels,columns=['tpot_lprofit_prediction'],index=fcpo_sprofit_testlabels.index),
            left_index=True,right_index=True).merge(
    pd.DataFrame(tpot_sprofit_test_predlabels,columns=['tpot_sprofit_prediction'],index=fcpo_sprofit_testlabels.index),
            left_index=True,right_index=True)

In [ ]:
fcpo_eval_df['lprofit_returns']=fcpo_eval_df.apply(lambda x: calculate_long_returns(x,'lprofit_ind','xgb_lprofit_prediction',60,25,0.0125,0.0125),axis=1)

fcpo_eval_df=fcpo_eval_df.assign(cummulative_lprofit_returns=fcpo_eval_df['lprofit_returns'].cumsum())

fcpo_eval_df['sprofit_returns']=fcpo_eval_df.apply(lambda x: calculate_short_returns(x,'sprofit_ind','xgb_sprofit_prediction',60,25,0.0125,0.0125),axis=1)
fcpo_eval_df=fcpo_eval_df.assign(cummulative_sprofit_returns=fcpo_eval_df['sprofit_returns'].cumsum())

fcpo_eval_df=fcpo_eval_df.assign(cummulative_lprofit_returns=fcpo_eval_df['cummulative_lprofit_returns']+10000,
            cummulative_sprofit_returns=fcpo_eval_df['cummulative_sprofit_returns']+10000)

In [ ]:
plt.figure(figsize=(15,10))
fcpo_eval_df['2017-01-01':'2018-10-01']['cummulative_lprofit_returns'].plot()
fcpo_eval_df['2017-01-01':'2018-10-01']['cummulative_sprofit_returns'].plot()
(fcpo_eval_df['2017-01-01':'2018-10-01']['cummulative_sprofit_returns']+
     fcpo_eval_df['2017-01-01':'2018-10-01']['cummulative_lprofit_returns']).plot()
plt.legend(['lprofit','sprofit','total'])

In [ ]:
lprofit_mdd_values=max_drawdown(fcpo_eval_df['2017-01-01':'2018-10-01']['cummulative_lprofit_returns'])

sprofit_mdd_values=max_drawdown(fcpo_eval_df['2017-01-01':'2018-10-01']['cummulative_sprofit_returns'])

total_mdd_values=max_drawdown(fcpo_eval_df['2017-01-01':'2018-10-01']['cummulative_sprofit_returns']+
                     fcpo_eval_df['2017-01-01':'2018-10-01']['cummulative_lprofit_returns'])

In [ ]:
print("Long Profit Model MDD:{mdd_value}, from {high_value} to {low_value}".format(mdd_value=round(lprofit_mdd_values[2],2)*100,
                                                                                  high_value=round(lprofit_mdd_values[0]),
                                                                                  low_value=round(lprofit_mdd_values[1])))

In [ ]:
print("Short Profit Model MDD:{mdd_value}, from {high_value} to {low_value}".format(mdd_value=round(sprofit_mdd_values[2],2)*100,
                                                                                  high_value=round(sprofit_mdd_values[0]),
                                                                                  low_value=round(sprofit_mdd_values[1])))

In [ ]:
print("Tota MDD:{mdd_value}, from {high_value} to {low_value}".format(mdd_value=round(total_mdd_values[2],2)*100,
                                                                                  high_value=round(total_mdd_values[0]),
                                                                                  low_value=round(total_mdd_values[1])))

## Auto ML

In [ ]:
import tpot


tpot_lclf=tpot.TPOTClassifier(verbosity=2,random_state=333,cv=fcpo_tsplit,scoring='roc_auc',
                             generations=50,population_size=50,offspring_size=50)
tpot_sclf=tpot.TPOTClassifier(verbosity=2,random_state=333,cv=fcpo_tsplit,scoring='roc_auc',
                             generations=50,population_size=50,offspring_size=50)

In [ ]:
fcpo_lprofit_trainlabels=fcpo_lprofit_trainlabels.applymap(lambda x: str(x))
fcpo_sprofit_trainlabels=fcpo_sprofit_trainlabels.applymap(lambda x: str(x))

In [ ]:
tpot_sclf.fit(fcpo_sprofit_traindata,fcpo_sprofit_trainlabels['lprofit_ind'])

In [ ]:
tpot_lclf.export('tpot_lclf.py')

### Exporting the Trading signal

In [ ]:
def derive_trading_signal(x):
    signal='None'
    if (x['rf_lprofit_prediction']==1 ) & (x['rf_sprofit_prediction']==0):
        signal='Long'
    elif(x['rf_lprofit_prediction']==0) & (x['rf_sprofit_prediction']==1):
        signal='Short'
    else:
        signal='None'
    return signal

trading_signal_df=fcpo_eval_df[['rf_lprofit_prediction','rf_sprofit_prediction']].apply(
    lambda x: derive_trading_signal(x),axis=1)

trading_signal_df.to_csv('trading_signal_2018.csv')